# Examples of Tracking and Curtain Visualization

This notebook utilizes concatenated data from the Pacific hake survey to create echograms and corresponding track and draped curtain visualizations. Additionally, it provides a demonstration of switching between two control modes: `echogram-control` mode and `track-control` mode. The hake survey data has been integrated with geographical coordinates for comprehensive analysis.

## The Significance of Tracking and Curtain Plotting

- **Echogram-Control Mode**: Fisheries scientists use this mode to visualize where the echosounder data, displayed as an echogram, was collected on the map. This helps them assess the fish's association with bathymetry.

- **Track-Control Mode**: In this mode, fisheries scientists can highlight a specific section of a ship track on the map and simultaneously view the corresponding echogram. This capability allows them to gain insights into the composition of fish and zooplankton at precise geographic locations.

- **Curtain Visualization**: Fisheries scientists can choose to represent longer sections of echograms or ship tracks on the map as curtains. This feature offers a more comprehensive view of how fish aggregations vary spatially.

- **Region of Interest Selection**: Fisheries scientists can select a specific area on the track display to focus on a particular region of interest. They can then examine the `Sv` (volume backscattering strength) within that selected area for in-depth fish analysis.

- **Exporting MVBS Dataset**: It's essential for fisheries scientists to export the `MVBS` (mean volume backscattering strength) dataset sliced by the selected box from the track. This exportation allows them to save the specific `Sv` values within that region to a separate file. These saved data can be further analyzed or shared with colleagues for collaborative research and insights.


## Import Packages and Data 

In [15]:
import panel
import xarray as xr

import echoshader

In [16]:
# from urllib import request

# # Calibratd data is stored in Google Drive
# url = 'https://drive.google.com/uc?export=download&id=197D0MW-bHaF6mZLcQwyr4zqyEHIfwsep'

# def urllib_download():
#     request.urlretrieve(url, 'concatenated_MVBS.nc')

# urllib_download() 

# Load sample data for testing
MVBS_ds = xr.open_mfdataset(
    paths="concatenated_MVBS.nc",
    data_vars="minimal",
    coords="minimal",
    combine="by_coords",
)

MVBS_ds

<xarray.Dataset>
Dimensions:            (channel: 4, echo_range: 150, ping_time: 875)
Coordinates:
  * channel            (channel) object 'GPT  18 kHz 009072058c8d 1-1 ES18-11...
  * echo_range         (echo_range) float64 0.0 5.0 10.0 ... 735.0 740.0 745.0
  * ping_time          (ping_time) datetime64[ns] 2017-07-24T19:30:00 ... 201...
    time1              (ping_time) datetime64[ns] dask.array<chunksize=(875,), meta=np.ndarray>
Data variables:
    Sv                 (channel, ping_time, echo_range) float64 dask.array<chunksize=(4, 875, 150), meta=np.ndarray>
    frequency_nominal  (channel) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    longitude          (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
    latitude           (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.7.1
    processing_time:              2023-05-30T17:40:45Z
    processing_function:          commongrid.compute_MVBS

## Track Demonstration in Track-Control Mode

Users have the flexibility to customize various settings for tracks in this demonstration:

- `tile`: This setting determines the map tile used for the track plot. You can explore different options in the gallery provided [here](https://holoviews.org/reference/elements/bokeh/Tiles.html).

- `control`: By default, this setting is set to False, activating track-control mode. However, you can set it to True to enable echogram-control mode.

In the example below, when using echogram-control mode, users have the capability to select specific areas on the echogram. Subsequently, the corresponding track will be displayed.

Here's a breakdown of the visual elements:
- A blue point signifies the starting point of the track.
- A red line depicts the course of the ship.

It's worth noting that when echo data originates from a moored platform, only one blue point will be displayed.


In [22]:
track = MVBS_ds.eshader.track(
    tile = 'OpenTopoMap',
)

eg = MVBS_ds.eshader.echogram(
        channel = ['GPT  18 kHz 009072058c8d 1-1 ES18-11',],
        cmap = "jet", 
)

panel.Column(track, eg)

BokehModel(combine_events=True, render_bundle={'docs_json': {'54bc4c54-5d2b-4f45-a358-8c1ed4909fbf': {'version…

There is only one control widget associated with the track:

- `tile_select`: This is a dropdown widget used to select the map tile. You can find further details and options in the [reference](https://holoviews.org/reference/elements/bokeh/Tiles.html).

In [18]:
tile_select = MVBS_ds.eshader.tile_select

track_panel = panel.Row(
    MVBS_ds.eshader.tile_select,
    track,
)

track_panel

BokehModel(combine_events=True, render_bundle={'docs_json': {'c3243871-3aa9-47c4-889e-529722a1f9b4': {'version…

## Track Demonstration in Echogram-Control Mode

Users have the flexibility to configure the behavior of the track by setting the `control` parameter during its initialization or by selecting control mode widgets to switch between echogram-control mode and track-control mode. To begin in echogram-control mode, simply set `control = True`.

In [23]:
control_mode_select = MVBS_ds.eshader.control_mode_select

eg_with_echogram_mode = MVBS_ds.eshader.echogram(
    channel = ['GPT  38 kHz 009072058146 2-1 ES38B',],
    cmap = "jet", 
)

track_with_echogram_mode = MVBS_ds.eshader.track(
    tile = 'OpenTopoMap',
    control = True,
)

panel.Column(
    control_mode_select,
    eg_with_echogram_mode,
    track_with_echogram_mode,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'1a08d970-492c-4fd0-8372-3032dfec23a2': {'version…

## Box Selection

Just like in `echogram-control mode`, users can utilize the `Box Select` feature in `track-control mode` to define a specific rectangular area on the track map and retrieve the corresponding dataset.

You can easily clear the selected box and reset the box selection by using the `Reset Button`.


In [24]:
data_from_box_select = MVBS_ds.eshader.get_data_from_box()

## Curtain Visualization

For sonar data collected from mobile platforms like ships, imagine the echogram as a curtain draped along the GPS track of the vessel.

In the `echogram-control` mode, the curtain aligns with the selected box area on the echograms.

In the `track-control` mode, the curtain corresponds to the chosen box area on the track.

Users have the ability to customize the default curtain settings with the following options:

- `channel`: Select the frequency channel to plot the curtain. The default values in the list are derived from the channel input of the echograms.

- `ratio`: Adjust the curtain ratio for z-axis spacing.

The colormap and Sv (volume backscattering strength) range of the curtain are determined by the values present in the echograms, which are equivalent to the values in the corresponding control widgets.

There are four control widgets associated with the curtain:

- `colormap`: A widget for controlling the colormap, shared with the echograms.

- `Sv_range_slider`: A slider widget for adjusting the Sv range, also shared with the echograms.

- `channel_select`: A dropdown widget for selecting the frequency channel.

- `curtain_ratio`: A numeric input widget for controlling the curtain ratio.

If you don't see the panel displayed, make sure to set up the 'pyvista' extension for `panel` by using the following code:

```python
panel.extension("pyvista")
```

Additionally, when working on an EC2 instance, configure the following settings for `pyvista`:
You can find why you should do this in the [reference](https://github.com/pyvista/pyvista/issues/177).
```python
pyvista.start_xvfb()
```

In [25]:
panel.extension("pyvista")

colormap = MVBS_ds.eshader.colormap

Sv_range_slider = MVBS_ds.eshader.Sv_range_slider

channel_select = MVBS_ds.eshader.channel_select

curtain_ratio = MVBS_ds.eshader.curtain_ratio

curtain = MVBS_ds.eshader.curtain()

curtain_panel = panel.Row(
    panel.Column(
        colormap,
        Sv_range_slider,
        channel_select,
        curtain_ratio,
    ),
    curtain,
)

curtain_panel

BokehModel(combine_events=True, render_bundle={'docs_json': {'6d379472-1b47-47ab-88fa-da04ce5e6498': {'version…

## Applying Plot Customizations

Similar to customizing echograms, users can input `Holoviews options` to apply personalized adjustments to the visualizations.

For detailed information about `Holoviews options`, please refer to this [link](https://holoviews.org/user_guide/Applying_Customizations.html#option-list-syntax).

When working with the curtain, users have the flexibility to input parameters such as `width = 600`, `height = 400`, and `orientation_widget = False`. For more details, you can explore the [reference](https://panel.holoviz.org/api/panel.pane.vtk.html#panel.pane.vtk.vtk.AbstractVTK).